In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import RandomEffects


df = pd.read_stata("cornwell.dta")
df = df.set_index(["county", "year"], drop=True)
df

crmrte    prbarr   prbconv   prbpris  avgsen     polpc  \
county year                                                             
1      81    0.039885  0.289696  0.402062  0.472222    5.61  0.001787   
       82    0.038345  0.338111  0.433005  0.506993    5.59  0.001767   
       83    0.030305  0.330449  0.525703  0.479705    5.80  0.001836   
       84    0.034726  0.362525  0.604706  0.520104    6.89  0.001886   
       85    0.036573  0.325395  0.578723  0.497059    6.55  0.001924   
...               ...       ...       ...       ...     ...       ...   
197    83    0.015575  0.226667  0.480392  0.428571    7.77  0.001073   
       84    0.013662  0.204188  1.410260  0.372727   10.11  0.001109   
       85    0.013086  0.180556  0.830769  0.333333    5.96  0.001054   
       86    0.012874  0.112676  2.250000  0.244444    7.68  0.001088   
       87    0.014193  0.207595  1.182930  0.360825   12.23  0.001186   

              density      taxpc  west  central  ...  lpctymle   lpctmin  \
county year                                      ...                       
1      81    2.307159  25.697630     0        1  ... -2.433870  3.006608   
       82    2.330254  24.874252     0        1  ... -2.449038  3.006608   
       83    2.341801  26.451443     0        1  ... -2.464036  3.006608   
       84    2.346420  26.842348     0        1  ... -2.478925  3.006608   
       85    2.364896  28.140337     0        1  ... -2.497306  3.006608   
...               ...        ...   ...      ...  ...       ...       ...   
197    83    0.869048  18.905853     1        0  ... -2.538060  1.697597   
       84    0.872024  22.704754     1        0  ... -2.548068  1.697597   
       85    0.875000  24.123611     1        0  ... -2.561072  1.697597   
       86    0.880952  24.981979     1        0  ... -2.580968  1.697597   
       87    0.889881  25.952581     1        0  ... -2.601006  1.697597   

             clcrmrte  clprbarr  clprbcon  clprbpri  clavgsen   clpolpc  \
county year                                                               
1      81         NaN       NaN       NaN       NaN       NaN       NaN   
       82   -0.039376  0.154542  0.074143  0.071048 -0.003571 -0.011364   
       83   -0.235316 -0.022922  0.193987 -0.055326  0.036879  0.038413   
       84    0.136180  0.092641  0.140006  0.080857  0.172213  0.026930   
       85    0.051825 -0.108054 -0.043918 -0.045320 -0.050606  0.020199   
...               ...       ...       ...       ...       ...       ...   
197    83   -0.148666 -0.010969 -0.127018  0.164303  0.157158  0.149330   
       84   -0.131037 -0.104441  1.076927 -0.139610  0.263255  0.032795   
       85   -0.043091 -0.123000 -0.529178 -0.111704 -0.528454 -0.050473   
       86   -0.016311 -0.471524  0.996334 -0.310156  0.253549  0.031580   
       87    0.097525  0.611073 -0.642936  0.389407  0.465272  0.086072   

              cltaxpc     clmix  
county year                      
1      81         NaN       NaN  
       82   -0.032565  0.030857  
       83    0.061477 -0.244732  
       84    0.014670 -0.027331  
       85    0.047223  0.172125  
...               ...       ...  
197    83    0.070461  0.020250  
       84    0.183103  0.026842  
       85    0.060617 -0.366374  
       86    0.034964 -0.067911  
       87    0.038116 -0.114410  

[630 rows x 57 columns]

In [3]:

# --- 0) carregar / preparar dados ---
# Suponho que já haja um DataFrame `df`. Se precisa carregar .dta:
# df = pd.read_stata("path/to/cornwell.dta")

# Ajuste: garantir colunas 'county' e 'year' existirem (ou extrair do índice)
if ("county" not in df.columns) or ("year" not in df.columns):
    # se o índice for MultiIndex (county, year)
    if isinstance(df.index, pd.MultiIndex):
        df = df.reset_index()  # traz county, year como colunas
    else:
        raise ValueError("DataFrame precisa ter colunas 'county' e 'year' ou ser MultiIndex (county,year).")

# variáveis do modelo
y_name = "lcrmrte"
x_names = ["lprbarr", "lprbconv", "lprbpris", "lavgsen", "lpolpc"]

# ordenar por county, year (opcional, só para clareza)
df = df.sort_values(["county", "year"]).reset_index(drop=True)

# N, T
N = df["county"].nunique()
T = df.groupby("county")["year"].nunique().mode().iloc[0]   # assume balanced panel; ajuste se necessary
K = len(x_names)  # número de regressores (sem constante)

# --- 1) ESTIMADOR WITHIN (FE) manual: demean por county e OLS das transformed vars ---
# calcular médias por county
means = df.groupby("county")[[y_name] + x_names].transform("mean")



In [4]:
# within (demeaned)
df_within = df[[y_name] + x_names] - means

# regressão OLS sem constante sobre as variáveis demeaned
y_within = df_within[y_name]
X_within = df_within[x_names]
fe_within = sm.OLS(y_within, X_within).fit()

# residuos within (usados para sigma_e)
resid_within = fe_within.resid

# --- 2) estimar sigma_e^2 (conforme fórmula do slide) ---
# grau de liberdade sugerido: N*(T-1) - K
df_dof_e = N * (T - 1) - K
sigma_e2 = (resid_within**2).sum() / df_dof_e
# garantir não-negatividade
sigma_e2 = float(sigma_e2)

# --- 3) BETWEEN estimator: regressão das médias por county ---
group_means = df.groupby("county")[[y_name] + x_names].mean()
Y_bar = group_means[y_name]
X_bar = group_means[x_names]

# adicionar constante ao between (o slide usa forma matricial; aqui usamos OLS com constante)
Xb = sm.add_constant(X_bar)
be_between = sm.OLS(Y_bar, Xb).fit()
resid_between = be_between.resid

# --- 4) estimar sigma_a^2 conforme slide ---
# SSR_between / (N - (K+1))  menos sigma_e2 / T
df_dof_a = N - (K + 1)   # se usamos constante no between, K+1 parâmetros
SSR_between = (resid_between**2).sum()
sigma_a2_hat = SSR_between / df_dof_a - sigma_e2 / T
# truncar se negativo
sigma_a2_hat = max(0.0, float(sigma_a2_hat))

# --- 5) calcular lambda (λ̂) conforme o slide: λ̂ = 1 - sqrt( σ_e^2 / (T σ_a^2 + σ_e^2) ) ---
den = sigma_e2 + T * sigma_a2_hat
if den <= 0:
    lam = 0.0
else:
    lam = 1.0 - np.sqrt(sigma_e2 / den)

# --- 6) transformar variáveis (quasi-GLS): y - λ y_bar_i , X - λ X_bar_i ---
# expandir as médias para o formato original (cada observação)
means_expanded = df.groupby("county")[[y_name] + x_names].transform("mean")
y_re = df[y_name] - lam * means_expanded[y_name]
X_re = df[x_names] - lam * means_expanded[x_names]

# adicionar constante (no RE a constante não some)
X_re_const = sm.add_constant(X_re)

# --- 7) OLS nas variáveis transformadas => estimador RE (GLS) manual ---
re_manual = sm.OLS(y_re, X_re_const).fit()

# --- 8) comparar com RandomEffects do linearmodels ---
# precisamos do painel index (county, year) para linearmodels
df_panel = df.set_index(["county", "year"])
re_lm = RandomEffects(df_panel[y_name], df_panel[x_names]).fit()

# --- 9) imprimir resultados e comparação ---
print("=== Resumo dos passos e estimativas ===")
print(f"N = {N}, T = {T}, K = {K}")
print(f"sigma_e^2 (from FE residuals) = {sigma_e2:.6g}")
print(f"sigma_a^2 (estimated from BETWEEN) = {sigma_a2_hat:.6g}")
print(f"lambda_hat = {lam:.6g}\n")

print("---- Coeficientes: RE manual (GLS via transformação) ----")
print(re_manual.params)
print("\n---- Coeficientes: RandomEffects (linearmodels) ----")
print(re_lm.params)

print("\nDiferença (manual - linearmodels):")
print(re_manual.params - re_lm.params)


=== Resumo dos passos e estimativas ===
N = 90, T = 7, K = 5
sigma_e^2 (from FE residuals) = 0.0215555
sigma_a^2 (estimated from BETWEEN) = 0.0898671
lambda_hat = 0.817982

---- Coeficientes: RE manual (GLS via transformação) ----
const      -0.351192
lprbarr    -0.448597
lprbconv   -0.346917
lprbpris   -0.187692
lavgsen     0.027629
lpolpc      0.418481
dtype: float64

---- Coeficientes: RandomEffects (linearmodels) ----
lprbarr    -0.434564
lprbconv   -0.400164
lprbpris   -0.129210
lavgsen    -0.061275
lpolpc      0.669860
Name: parameter, dtype: float64

Diferença (manual - linearmodels):
const            NaN
lavgsen     0.088905
lpolpc     -0.251379
lprbarr    -0.014033
lprbconv    0.053247
lprbpris   -0.058482
dtype: float64
